### ML with Pyspark
+ classify/predict winning team 

### Data Source
+ https://www.kaggle.com/hikne707/big-five-european-soccer-leagues 

In [3]:
# Load our Packages
from pyspark import SparkContext

In [6]:
sc = SparkContext(master='local[2]')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at C:\Users\ngand\anaconda3\lib\site-packages\IPython\utils\py3compat.py:168 

In [5]:
# spark UI
sc.stop

<bound method SparkContext.stop of <SparkContext master=local[*] appName=PySparkShell>>

In [1]:
# load packages
from pyspark.sql import Sparksession

ImportError: cannot import name 'Sparksession' from 'pyspark.sql' (D:\Learning\pyspark\spark-3.1.1-bin-hadoop2.7\python\pyspark\sql\__init__.py)